# Pandas for Exploratory Data Analysis II 

Pandas a very useful Python library for data manipulation and exploration. We have so much more to see!

In this lesson, we'll continue exploring Pandas for EDA. Specifically: 

- Identify and handle missing values with Pandas.
- Implement groupby statements for specific segmented analysis.
- Use apply functions to clean data with Pandas.

We'll implicitly review many functions from our first Pandas lesson along the way!

## Remember the AdventureWorks Cycles Dataset?
<img align="right" src="http://lh6.ggpht.com/_XjcDyZkJqHg/TPaaRcaysbI/AAAAAAAAAFo/b1U3q-qbTjY/AdventureWorks%20Logo%5B5%5D.png?imgmax=800">

Here's the Production.Product table [data dictionary](https://www.sqldatadictionary.com/AdventureWorks2014/Production.Product.html), which is a description of the fields (columns) in the table (the .csv file we will import below):<br>
- **ProductID** - Primary key for Product records.
- **Name** - Name of the product.
- **ProductNumber** - Unique product identification number.
- **MakeFlag** - 0 = Product is purchased, 1 = Product is manufactured in-house.
- **FinishedGoodsFlag** - 0 = Product is not a salable item. 1 = Product is salable.
- **Color** - Product color.
- **SafetyStockLevel** - Minimum inventory quantity.
- **ReorderPoint** - Inventory level that triggers a purchase order or work order.
- **StandardCost** - Standard cost of the product.
- **ListPrice** - Selling price.
- **Size** - Product size.
- **SizeUnitMeasureCode** - Unit of measure for the Size column.
- **WeightUnitMeasureCode** - Unit of measure for the Weight column.
- **DaysToManufacture** - Number of days required to manufacture the product.
- **ProductLine** - R = Road, M = Mountain, T = Touring, S = Standard
- **Class** - H = High, M = Medium, L = Low
- **Style** - W = Womens, M = Mens, U = Universal
- **ProductSubcategoryID** - Product is a member of this product subcategory. Foreign key to ProductSubCategory.ProductSubCategoryID.
- **ProductModelID** - Product is a member of this product model. Foreign key to ProductModel.ProductModelID.
- **SellStartDate** - Date the product was available for sale.
- **SellEndDate** - Date the product was no longer available for sale.
- **DiscontinuedDate** - Date the product was discontinued.
- **rowguid** - ROWGUIDCOL number uniquely identifying the record. Used to support a merge replication sample.
- **ModifiedDate** - Date and time the record was last updated.


### Import Pandas

In [7]:
import pandas as pd
import numpy as np # used for linear algebra and random sampling
# used for plotting charts within the notebook (instead of a separate window)
%matplotlib inline

### Read in the dataset

We are using the `read_csv()` method (and the `\t` separator to specify tab-delimited columns).

In [13]:
# read in the dataset
prod_df = pd.read_csv('./data/Production.Product.csv', sep='\t')

In [10]:
# let's check out the first 3 rows again, for old time's sake
prod_df.head(3)

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,...,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
0,1,Adjustable Race,AR-5381,0,0,NaN,1000,750,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,2008-04-30 00:00:00,NaN,NaN,{694215B7-08F7-4C0D-ACB1-D734BA44C0C8},2014-02-08 10:01:36.827000000
1,2,Bearing Ball,BA-8327,0,0,NaN,1000,750,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,2008-04-30 00:00:00,NaN,NaN,{58AE3C20-4F3A-4749-A7D4-D568806CC537},2014-02-08 10:01:36.827000000
2,3,BB Ball Bearing,BE-2349,1,0,NaN,800,600,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,2008-04-30 00:00:00,NaN,NaN,{9C21AED2-5BFA-4F18-BCB8-F11638DC2E4E},2014-02-08 10:01:36.827000000


In [11]:
# and the number of rows x cols
prod_df.shape

(504, 25)

### Reset our index (like last time)

Let's bring our `ProductID` column into the index since it's the PK (primary key) of our table and that's where PKs belong as a best practice.

In [14]:
# repalace auto-generated index with the ProductID column
prod_df.set_index('ProductID', inplace=True)

## Handling missing data

Recall missing data is a systemic, challenging problem for data scientists. Imagine conducting a poll, but some of the data gets lost, or you run out of budget and can't complete it! 😮<br><br>

"Handling missing data" itself is a broad topic. We'll focus on two components:

- Using Pandas to identify we have missing data
- Strategies to fill in missing data (known in the business as `imputing`)
- Filling in missing data with Pandas


### Identifying missing data

Before *handling*, we must identify we're missing data at all!

We have a few ways to explore missing data, and they are reminiscient of our Boolean filters.

In [15]:
# True when data is missing #true means there is a no na in there and falls mean to count na in there
prod_df.isna().head(10)

,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,...,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
ProductID,,,,,,,,,,,,,,,,,,,,,
1,False,False,False,False,True,False,False,False,False,True,...,True,True,True,True,True,False,True,True,False,False
2,False,False,False,False,True,False,False,False,False,True,...,True,True,True,True,True,False,True,True,False,False
3,False,False,False,False,True,False,False,False,False,True,...,True,True,True,True,True,False,True,True,False,False
4,False,False,False,False,True,False,False,False,False,True,...,True,True,True,True,True,False,True,True,False,False
316,False,False,False,False,True,False,False,False,False,True,...,True,True,True,True,True,False,True,True,False,False
317,False,False,False,False,False,False,False,False,False,True,...,True,False,True,True,True,False,True,True,False,False
318,False,False,False,False,False,False,False,False,False,True,...,True,False,True,True,True,False,True,True,False,False
319,False,False,False,False,False,False,False,False,False,True,...,True,True,True,True,True,False,True,True,False,False
320,False,False,False,False,False,False,False,False,False,True,...,True,True,True,True,True,False,True,True,False,False


In [16]:
# True when data is missing
prod_df.isna().head(3)

,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,...,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
ProductID,,,,,,,,,,,,,,,,,,,,,
1,False,False,False,False,True,False,False,False,False,True,...,True,True,True,True,True,False,True,True,False,False
2,False,False,False,False,True,False,False,False,False,True,...,True,True,True,True,True,False,True,True,False,False
3,False,False,False,False,True,False,False,False,False,True,...,True,True,True,True,True,False,True,True,False,False


Now, we may want to see null values in aggregate. We can use `sum()` to sum down a given column

In [17]:
# here is a quick and dirty way to do it
prod_df.isna().sum()

Name                       0
ProductNumber              0
MakeFlag                   0
FinishedGoodsFlag          0
Color                    248
SafetyStockLevel           0
ReorderPoint               0
StandardCost               0
ListPrice                  0
Size                     293
SizeUnitMeasureCode      328
WeightUnitMeasureCode    299
Weight                   299
DaysToManufacture          0
ProductLine              226
Class                    257
Style                    293
ProductSubcategoryID     209
ProductModelID           209
SellStartDate              0
SellEndDate              406
DiscontinuedDate         504
rowguid                    0
ModifiedDate               0
dtype: int64

In [18]:
#here the info tell you how many values are not missing
prod_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 504 entries, 1 to 999
Data columns (total 24 columns):
Name                     504 non-null object
ProductNumber            504 non-null object
MakeFlag                 504 non-null int64
FinishedGoodsFlag        504 non-null int64
Color                    256 non-null object
SafetyStockLevel         504 non-null int64
ReorderPoint             504 non-null int64
StandardCost             504 non-null float64
ListPrice                504 non-null float64
Size                     211 non-null object
SizeUnitMeasureCode      176 non-null object
WeightUnitMeasureCode    205 non-null object
Weight                   205 non-null float64
DaysToManufacture        504 non-null int64
ProductLine              278 non-null object
Class                    247 non-null object
Style                    211 non-null object
ProductSubcategoryID     295 non-null float64
ProductModelID           295 non-null float64
SellStartDate            504 non-null ob

Look! We've found missing values!

How could this missing data be problematic for our analysis?

### Understanding missing data

Finding missing data is the easy part! Determining way to do next is more complicated.

Typically, we are most interested in knowing **why** we are missing data. Once we know what 'type of missingness' we have (the source of missing data), we can proceed effectively.

Let's first quantify how much data we are missing. Here is another implementation of `prod_df.isnull().sum()`, only wrapped with a `DataFrame` and some labels to make it a little more user-friendly:

In [19]:
# or we can make things pretty as follows
null_df = pd.DataFrame(prod_df.isna().sum(), columns=['Count of Nulls'])
null_df.index.name = 'Column'
null_df.sort_values(['Count of Nulls'], ascending=False)

,Count of Nulls
Column,
DiscontinuedDate,504
SellEndDate,406
SizeUnitMeasureCode,328
Weight,299
WeightUnitMeasureCode,299
Size,293
Style,293
Class,257
Color,248


### Filling in missing data

How we fill in data depends largely on why it is missing (types of missingness) and what sampling we have available to us.

We may:

- Delete missing data altogether
- Fill in missing data with:
    - The average of the column
    - The median of the column
    - A predicted amount based on other factors
- Collect more data:
    - Resample the population
    - Followup with the authority providing data that is missing


In our case, let's focus on handling missing values in `Color`. Let's get a count of the unique values in that column. We will need to use the `dropna=False` kwarg, otherwise the `pd.Series.value_counts()` method will not count `NaN` (null) values.

In [20]:
# let's get a value count with the nulls included
prod_df['Color'].value_counts(dropna=False)

NaN             248
Black            93
Silver           43
Red              38
Yellow           36
Blue             26
Multi             8
Silver/Black      7
White             4
Grey              1
Name: Color, dtype: int64

We have 248 nulls!

Option 1: Drop the missing values.

In [21]:
# drops rows where any row has a missing value - this does not happen *in place*, so we are not actually dropping
prod_df['Color'].dropna(inplace=False)

ProductID
317     Black
318     Black
319     Black
320    Silver
321    Silver
322     Black
332    Silver
351    Silver
352    Silver
461    Silver
679    Silver
680     Black
706       Red
707       Red
708     Black
709     White
710     White
711      Blue
712     Multi
713     Multi
714     Multi
715     Multi
716     Multi
717       Red
718       Red
719       Red
720       Red
721       Red
722     Black
723     Black
        ...  
967      Blue
968      Blue
969      Blue
970      Blue
971      Blue
972      Blue
973    Yellow
974    Yellow
975    Yellow
976    Yellow
977     Black
978      Blue
979      Blue
980    Silver
981    Silver
982    Silver
983    Silver
984    Silver
985    Silver
986    Silver
987    Silver
988    Silver
989     Black
990     Black
991     Black
992     Black
993     Black
997     Black
998     Black
999     Black
Name: Color, Length: 256, dtype: object

**Important!** `pd.DataFrame.dropna()` and `pd.Series.dropna()` are very versatile! Let's look at the docs (Series is similar):

```python
Signature: pd.DataFrame.dropna(self, axis=0, how='any', thresh=None, subset=None, inplace=False)
Docstring:
Remove missing values.

See the :ref:`User Guide <missing_data>` for more on which values are
considered missing, and how to work with missing data.

Parameters
----------
axis : {0 or 'index', 1 or 'columns'}, default 0
    Determine if rows or columns which contain missing values are
    removed.

    * 0, or 'index' : Drop rows which contain missing values.
    * 1, or 'columns' : Drop columns which contain missing value.

    .. deprecated:: 0.23.0: Pass tuple or list to drop on multiple
    axes.
how : {'any', 'all'}, default 'any'
    Determine if row or column is removed from DataFrame, when we have
    at least one NA or all NA.

    * 'any' : If any NA values are present, drop that row or column.
    * 'all' : If all values are NA, drop that row or column.
thresh : int, optional
    Require that many non-NA values.
subset : array-like, optional
    Labels along other axis to consider, e.g. if you are dropping rows
    these would be a list of columns to include.
inplace : bool, default False
    If True, do operation inplace and return None.
```

**how**: This tells us if we want to remove a row if _any_ of the columns have a null, or _all_ of the columns have a null.<br>
**subset**: We can input an array here, like `['Color', 'Size', 'Weight']`, and it will only consider nulls in those columns. This is very useful!<br>
**inplace**: This is if you want to mutate (change) the source dataframe. Default is `False`, so it will return a _copy_ of the source dataframe.

To accomplish the same thing, but implement it on our entire dataframe, we can do the following:

In [22]:
# drops all nulls from the Color column, but returns the entire dataframe instead of just the Color column
prod_df.dropna(how='any', subset=['Color'], inplace=False)

Column,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,...,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
ProductID,,,,,,,,,,,,,,,,,,,,,
317,LL Crankarm,CA-5965,0,0,Black,500,375,0.0000,0.00,NaN,...,NaN,L,NaN,NaN,NaN,2008-04-30 00:00:00,NaN,NaN,{3C9D10B7-A6B2-4774-9963-C19DCEE72FEA},2014-02-08 10:01:36.827000000
318,ML Crankarm,CA-6738,0,0,Black,500,375,0.0000,0.00,NaN,...,NaN,M,NaN,NaN,NaN,2008-04-30 00:00:00,NaN,NaN,{EABB9A92-FA07-4EAB-8955-F0517B4A4CA7},2014-02-08 10:01:36.827000000
319,HL Crankarm,CA-7457,0,0,Black,500,375,0.0000,0.00,NaN,...,NaN,NaN,NaN,NaN,NaN,2008-04-30 00:00:00,NaN,NaN,{7D3FD384-4F29-484B-86FA-4206E276FE58},2014-02-08 10:01:36.827000000
320,Chainring Bolts,CB-2903,0,0,Silver,1000,750,0.0000,0.00,NaN,...,NaN,NaN,NaN,NaN,NaN,2008-04-30 00:00:00,NaN,NaN,{7BE38E48-B7D6-4486-888E-F53C26735101},2014-02-08 10:01:36.827000000
321,Chainring Nut,CN-6137,0,0,Silver,1000,750,0.0000,0.00,NaN,...,NaN,NaN,NaN,NaN,NaN,2008-04-30 00:00:00,NaN,NaN,{3314B1D7-EF69-4431-B6DD-DC75268BD5DF},2014-02-08 10:01:36.827000000
322,Chainring,CR-7833,0,0,Black,1000,750,0.0000,0.00,NaN,...,NaN,NaN,NaN,NaN,NaN,2008-04-30 00:00:00,NaN,NaN,{F0AC2C4D-1A1F-4E3C-B4D9-68AEA0EC1CE4},2014-02-08 10:01:36.827000000
332,Freewheel,FH-2981,0,0,Silver,500,375,0.0000,0.00,NaN,...,NaN,NaN,NaN,NaN,NaN,2008-04-30 00:00:00,NaN,NaN,{D864879A-E8B1-4F7B-BAFA-1F136089C2C8},2014-02-08 10:01:36.827000000
351,Front Derailleur Cage,FC-3982,0,0,Silver,800,600,0.0000,0.00,NaN,...,NaN,NaN,NaN,NaN,NaN,2008-04-30 00:00:00,NaN,NaN,{01C901E3-4323-48ED-AB9E-9BFDA28BDEF6},2014-02-08 10:01:36.827000000
352,Front Derailleur Linkage,FL-2301,0,0,Silver,800,600,0.0000,0.00,NaN,...,NaN,NaN,NaN,NaN,NaN,2008-04-30 00:00:00,NaN,NaN,{88ED2E08-E775-4915-B506-831600B773FD},2014-02-08 10:01:36.827000000


Option 2: Fill in missing values

Traditionally, we fill missing data with a median, average, or mode (most frequently occurring). For `Color`, let's replace the nulls with the string value `NoColor`.

Let's first look at the way we'd do it with a single column, using the `pd.Series.fillna()` method:

In [23]:
prod_df['Color'].fillna(value='NoColor').head(10)

ProductID
1      NoColor
2      NoColor
3      NoColor
4      NoColor
316    NoColor
317      Black
318      Black
319      Black
320     Silver
321     Silver
Name: Color, dtype: object

In [ ]:
prod_df.head()

Now let's see how we'd do it to the whole dataframe, using the `pd.DataFrame.fillna()` method. Notice the similar API between the methods with the `value` kwarg. Good congruent design, pandas development team! The full dataframe is returned, and not just a column.

In [ ]:
prod_df.fillna(value={'Color': 'NoColor'})


But wait! There's more! We can reference any other data or formulas we want with the imputation (the value we fill the nulls with). This is very handy if you want to impute with the average or median of that column... or even another column altogether! Here is an example where we will replace the nulls of `Color` with the average value from the `ListPrice` column. This has no practical value in this application, but immense value in other applications.

In [24]:
df = prod_df.fillna(value={'Color': prod_df['ListPrice'].mean()}).head(3)

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3 entries, 1 to 3
Data columns (total 24 columns):
Name                     3 non-null object
ProductNumber            3 non-null object
MakeFlag                 3 non-null int64
FinishedGoodsFlag        3 non-null int64
Color                    3 non-null object
SafetyStockLevel         3 non-null int64
ReorderPoint             3 non-null int64
StandardCost             3 non-null float64
ListPrice                3 non-null float64
Size                     0 non-null object
SizeUnitMeasureCode      0 non-null object
WeightUnitMeasureCode    0 non-null object
Weight                   0 non-null float64
DaysToManufacture        3 non-null int64
ProductLine              0 non-null object
Class                    0 non-null object
Style                    0 non-null object
ProductSubcategoryID     0 non-null float64
ProductModelID           0 non-null float64
SellStartDate            3 non-null object
SellEndDate              0 non-null obj

They're gone! Important points:

- Don't forget to use the `inplace=True` kwarg to mutate the source dataframe (i.e. 'save changes'). 
- It is helpful to not use `inplace=True` initially to ensure your code/logic is correct, prior to making permanent changes.

### Exercise (not in the original notebook)
Replace the null values in ```ProductLine``` with ```"NoProdLine"```

In [ ]:
# answer here


## Groupby Statements

In Pandas, groupby statements are similar to pivot tables in that they allow us to segment our population to a specific subset.

For example, if we want to know the average number of bottles sold and pack sizes per city, a groupby statement would make this task much more straightforward.


To think how a groupby statement works, think about it like this:

- **Split:** Separate our DataFrame by a specific attribute, for example, group by `Color`
- **Combine:** Put our DataFrame back together and return some _aggregated_ metric, such as the `sum`, `count`, or `max`.

![](http://i.imgur.com/yjNkiwL.png)

Let's try it out!

Let's group by `Color`, and get a count of products for each color.

In [26]:
# group by Color, giving the number of products of each %color
prod_df.groupby(by=['Color']).count().reset_index()

Column,Color,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,...,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
0,Black,93,93,93,93,93,93,93,93,71,...,86,72,71,89,89,93,44,0,93,93
1,Blue,26,26,26,26,26,26,26,26,25,...,26,22,25,26,26,26,0,0,26,26
2,Grey,1,1,1,1,1,1,1,1,0,...,1,0,0,1,1,1,1,0,1,1
3,Multi,8,8,8,8,8,8,8,8,7,...,8,0,8,8,8,8,3,0,8,8
4,Red,38,38,38,38,38,38,38,38,37,...,38,37,37,38,38,38,30,0,38,38
5,Silver,43,43,43,43,43,43,43,43,31,...,31,30,30,36,36,43,6,0,43,43
6,Silver/Black,7,7,7,7,7,7,7,7,0,...,7,6,0,7,7,7,0,0,7,7
7,White,4,4,4,4,4,4,4,4,4,...,4,0,4,4,4,4,2,0,4,4
8,Yellow,36,36,36,36,36,36,36,36,36,...,36,32,36,36,36,36,0,0,36,36


What do we notice about this output? Are all columns the same? Why or why not?

We can see that the `.count()` method excludes nulls, and there is no way to change this with the current implementation:
```python
Signature: .count()
Docstring: Compute count of group, excluding missing values 
File:      ~/miniconda3/envs/ga/lib/python3.7/site-packages/pandas/core/groupby/groupby.py
Type:      method
```

As a best practice, you should either:
- fill in nulls prior to your .count(), or
- use the PK (primary key) of the table, which is guaranteed non-null

In [27]:
# here we can use 'x' as a dummy placeholder for nulls, simply to get consistent counts for all columns
prod_df.fillna(value={'Color': 'x'}).groupby('Color', as_index=False).count()

Column,Color,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,...,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
0,Black,93,93,93,93,93,93,93,93,71,...,86,72,71,89,89,93,44,0,93,93
1,Blue,26,26,26,26,26,26,26,26,25,...,26,22,25,26,26,26,0,0,26,26
2,Grey,1,1,1,1,1,1,1,1,0,...,1,0,0,1,1,1,1,0,1,1
3,Multi,8,8,8,8,8,8,8,8,7,...,8,0,8,8,8,8,3,0,8,8
4,Red,38,38,38,38,38,38,38,38,37,...,38,37,37,38,38,38,30,0,38,38
5,Silver,43,43,43,43,43,43,43,43,31,...,31,30,30,36,36,43,6,0,43,43
6,Silver/Black,7,7,7,7,7,7,7,7,0,...,7,6,0,7,7,7,0,0,7,7
7,White,4,4,4,4,4,4,4,4,4,...,4,0,4,4,4,4,2,0,4,4
8,Yellow,36,36,36,36,36,36,36,36,36,...,36,32,36,36,36,36,0,0,36,36
9,x,248,248,248,248,248,248,248,248,0,...,41,48,0,50,50,248,12,0,248,248


In [ ]:
prod_df[['Color', 'ListPrice']].head()

Let's find out the most expensive price for an item, by `Color`:

In [29]:
# Notice we use a list to reference the multiple columns.
prod_df[['Color', 'ListPrice']].groupby(by='Color').max().sort_values('ListPrice', ascending=False).reset_index()

# apply a reset_index() to remove the multi-index 


Column,Color,ListPrice
0,Red,3578.27
1,Silver,3399.99
2,Black,3374.99
3,Blue,2384.07
4,Yellow,2384.07
5,Grey,125.00
6,Multi,89.99
7,Silver/Black,80.99
8,White,9.50


We can also do multi-level groupbys. This is referred to as a `Multiindex` dataframe. Here, we can see the following fields in a nested group by, with a count of Name (with nulls filled!); effectively giving us a count of the number of products for every unique Class/Style combination:

- Class - H = High, M = Medium, L = Low
- Style - W = Womens, M = Mens, U = Universal

In [34]:
prod_df[['Color', 'ListPrice']].groupby(by='Color').max().sort_values('ListPrice', ascending=False)

Column,ListPrice
Color,
Red,3578.27
Silver,3399.99
Black,3374.99
Blue,2384.07
Yellow,2384.07
Grey,125.00
Multi,89.99
Silver/Black,80.99
White,9.50


In [ ]:
df = prod_df.fillna(value={'Name': 'x'}).groupby(by=['Class', 'Style']).count()[['Name']]
df

#### Aggregating Method
```df.groupby('<grouping column>').agg({'<aggregating column>':'<aggregating function>'})```

The other form is:
```df.groupby('<grouping column>').agg({ [<aggregating function>] })```

    
We can also use the `.agg()` method with multiple arguments, to simulate a `.describe()` method like we used before:

In [ ]:
prod_df.groupby(by=['Color'])['ListPrice',  'StandardCost'].agg(['count', 'mean', 'min', 'max', 'median']).reset_index()

In [ ]:
gb = prod_df.groupby(by=['Color'])['ListPrice', 'StandardCost']

#.agg({'ListPrice': ['mean', 'median', 'var'], 'StandardCost': 'max'}).reset_index()

In [ ]:
is_black = gb.agg({'ListPrice': ['mean', 'median', 'var'], 'StandardCost': 'max'}).reset_index()['Color'] == 'Black'


gb.agg({'ListPrice': ['mean', 'median', 'var'], 'StandardCost': 'max'}).reset_index()[is_black]

In [ ]:
prod_df.head(2).T

#### Exercise
Write a groupby statement for at least 3 numerical columns and apply a different aggregation function to each. Be prepared to explain what you did.

## Apply functions for column operations

Apply functions allow us to perform a complex operation across an entire columns highly efficiently.

For example, let's say we want to change our colors from a word, to just a single letter. How would we do that?

The first step is writing a function, with the argument being the value we would receive from each cell in the column. This function will mutate the input, and return the result. This result will then be _applied_ to the source dataframe (if desired).

In [ ]:
# get a list of all unique colors
prod_df['Color'].unique()

In [35]:
# create a function to convert colors to letters
def color_to_letter(color): 
    color_dict = {
        'Black': 'B',
        'Silver': 'S',
        'Red': 'R',
        'White': 'W',
        'Blue': 'L',
        'Multi': 'M',
        'Yellow': 'Y',
        'Grey': 'G',
        'Silver/Black': 'V'
    }
    
    try:
        return color_dict[color]
    except:
        return 'N'

In [36]:
assert color_to_letter('Blue') == 'L'

In [37]:
prod_df['Color'].tail()

ProductID
995      NaN
996      NaN
997    Black
998    Black
999    Black
Name: Color, dtype: object

Now we can _apply_ this function to our `pd.Series` object, returning the result (which we can use to overwrite the source, if we choose).

In [ ]:
prod_df['Color'].apply(color_to_letter).tail()

The `pd.DataFrame.apply` implementation is similar, however it effectively 'scrolls through' the columns and passes each one sequentially to your function:

```python
Objects passed to the function are Series objects whose index is
either the DataFrame's index (``axis=0``) or the DataFrame's columns
(``axis=1``).
```

It should only be used when you wish to apply the same function to all columns (or rows) of your `pd.DataFrame` object.

We can also use `pd.Series.apply()` with a **labmda expression**. This is an undeclared function and is commonly used for simple functions within the `.apply()` method. Let's use it to add $100 to our `ListPrice` column. Hey, baby needs new shoes!

In [ ]:
# prior to apply method
prod_df['ListPrice'].tail(10)

In [ ]:
# and now with 100 more dollars!
f = lambda x: x + 100


prod_df['dumm'] = prod_df['ListPrice'].apply(f)

In [ ]:
prod_df['dummy'] = 10

In [ ]:
prod_df.tail()

Boom! Maybe financing that new boat wasn't such a bad idea after all!

**Your turn:** Identify one other column where we may want to write a new apply function, or use the one we just created for the purposes of cleaning up our dataset.

In [ ]:
0# identify a column to change


In [ ]:
# write a function to change that column (or columns) note: if using a lambda function, you can leave this blank


In [ ]:
# apply that function across the whole column


## Wrap up

We've covered even more useful information! Here are the key takeaways:

- **Missing data** comes in many shapes and sizes. Before deciding how to handle it, we identify it exists. We then derive how the missingness is affecting our dataset, and make a determination about how to fill in values.

```python
# pro tip for identifying missing data
df.isna().sum()
```

- **Grouby** statements are particularly useful for a subsection-of-interest analysis. Specifically, zooming in on one condition, and determining relevant statstics.

```python
# group by 
df.groupby('column').agg['count', 'mean', 'max', 'min']
```

- **Apply functions** help us clean values across an entire DataFrame column. They are *like* a for loop for cleaning, but many times more efficient. They follow a common pattern:
1. Write a function that works on a single value
2. Test that function on a single value
3. Apply that function to a whole column

(The most confusing part of apply functions is that we write them with *a single value* in mind, and then apply them to many single values at once.)